In [1]:
import pandas as pd
import africastalking
import re
from stac_rainfall import load_chirps_data

from datetime import datetime, date

In [5]:
# Initialize SDK
#sender = " " # use your username from AfricasTalking 
#username = " " # use your username from AfricasTalking 
#api_key = " "       # use your API key

# Initialize SDK
sender = "AGRIBORA"
username = "farmer_subscription"    # use 'sandbox' for development in the test environment
api_key = "0b8431b6934159dba02fee7fd359004f345f4679052f4009f86e8a3f7d11ff7a"       # use your sandbox app API key for development in the test environment

# Initialize services
africastalking.initialize(username, api_key)
sms = africastalking.SMS

In [2]:
sms_df = pd.read_excel(r'data/Decision tree_v1.0.xlsx')
farmers_df = pd.read_excel(r'data/smartfarmer_contacts.xlsx')
maize_df = sms_df.query('Crop == "Maize"')
soybeans_df = sms_df.query('Crop == "Soybeans"')

In [7]:
def get_dap(farmer_id):
    # here we get the current day anfter planting based on the planting date
    datetime_str = farmers_df.planting_date.iloc[farmer_id]
    datetime_object = datetime.strptime(datetime_str, '%d.%m.%Y')
    dt = datetime.today()
    if datetime_object.year == dt.year:
        dap = datetime_object.timetuple().tm_yday - dt.timetuple().tm_yday
    elif datetime_object.year < dt.year:
        delta = (date(2021, 12, 31) - datetime_object.date()).days
        dap = dt.timetuple().tm_yday + delta
    else:
        print("Date cannot be in the future!")
    return dap

def get_growthStage(dap):
    # returns the stage of development for the given day after planting
    enum_list = maize_df.DAP.unique().tolist()
    for num in enum_list:
        if dap < num:
            break
    idx = enum_list.index(num)
    return maize_df[maize_df['DAP']== enum_list[idx-1]].Stage.unique().tolist()[0]

def get_SMS(farmer_id, precip, temp):
    # Compares the available precipitation amount with the decision table
    # returns a dataframe with potential messages for the specific rainfall amount
    
    dap = get_dap(farmer_id)
    stage = get_growthStage(dap)
    stage_df = maize_df[maize_df['Stage']== stage] # get df with a particular stage
    unique_params = stage_df.Precipitation.str.extract('(\d+)')[0].unique().tolist()
    min_val = stage_df.Precipitation.iloc[0]
    max_val = stage_df.Precipitation.iloc[-1]

    if precip < int(unique_params[0]):
        df = stage_df.query('Precipitation == @min_val')
    elif precip > int(unique_params[-1]):
        df = stage_df.query('Precipitation == @max_val')
    else:
        df = stage_df.query('Precipitation != @min_val & Precipitation != @max_val') 
    prec_df = df
    temp_params = prec_df.Temp.str.extract('(\d+)')[0].unique().tolist()
    min_val = prec_df.Temp.iloc[0]
    max_val = prec_df.Temp.iloc[-1]

    if temp < int(temp_params[0]):
        temp_df = prec_df.query('Temp == @min_val')
    elif temp > int(temp_params[-1]):
        temp_df = prec_df.query('Temp == @max_val')
    else:
        temp_df = prec_df.query('Temp != @min_val & Temp != @max_val')   
    return temp_df.SMS.item()

def send_SMS(farmer_id):
    msg = get_SMS(farmer_id, 33, 14)
    number = '+' + str(farmers_df.Phone_number.iloc[farmer_id])
    success = 0
    sent = 0
    response = sms.send(msg, [number], sender) # send SMS
    sent +=1 # record that SMS was sent
    if response['SMSMessageData']['Recipients'][0]['status'] == 'Success': # check if message was sent successfully
        success +=1
    print(sent)
    print(success) 

In [8]:
get_SMS(1, 33, 22)
#send_SMS(0)

'Little rain and normal temperature. Grain yield may reduce. Check for dark brown, powdery masses appearing as charcoal dust on maize, if found remove and bury 50 cm deep in soil.'

In [14]:
farmers_df

,Name,Phone_number,planting_date,Crop,Variety,latitude,longitude
0,Kizito Odhiambo,254705859648,15.10.2021,Maize,H84,0.735884,34.5833
1,Grace Jepkosgei,254733631203,03.11.2021,Soybeans,SC Saga,0.055216,34.2770


In [9]:
start_date = '2020-01-01'
end_date = '2020-01-31'
#bbox = [34.1, 0.72, 34.2, 0.73]
bbox = [40, -2.2, 40.5, -2.3]
lat,lon = 40, -2.2#farmers_df.latitude.iloc[1],farmers_df.longitude.iloc[1]
chirps_dataset = load_chirps_data(lat,lon, start_date, end_date, bbox)

Found: 31 datasets


In [10]:
chirps_dataset

<xarray.Dataset>
Dimensions:      (time: 31, latitude: 0, longitude: 2)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01T11:59:59.500000 ... 2020-01...
  * latitude     (latitude) float64 
  * longitude    (longitude) float64 34.12 34.18
    spatial_ref  int32 4326
Data variables:
    rainfall     (time, latitude, longitude) float32 dask.array<chunksize=(1, 0, 2), meta=np.ndarray>
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

In [12]:
chirps_dataset.rainfall#.mean(['x', 'y']).plot.line('b-^', figsize=(11,4))

<xarray.DataArray 'rainfall' (time: 366, latitude: 0, longitude: 0)>
dask.array<where, shape=(366, 0, 0), dtype=float32, chunksize=(1, 0, 0), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01T11:59:59.500000 ... 2020-12...
  * latitude     (latitude) float64 
  * longitude    (longitude) float64 
    spatial_ref  int32 4326
Attributes:
    nodata:        -9999
    units:         mm
    crs:           EPSG:4326
    grid_mapping:  spatial_ref